# 'Phil's Chocolates' Profit Dashboard using MySQL queries.

**Import Libraries.**

In [1]:
import numpy as np
import pandas as pd
import pymysql
from pymysql import connect
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

**STEP 1. Set up Python connection to MySQL using pymysql.**

In [2]:
db = connect(
    host = 'localhost',
    user = 'root',
    passwd = 'Habits98!',
    database = 'phils_chocolates')

**STEP 2: Test MySQL queries.**

In [3]:
cur = db.cursor()
query = ("SELECT \
             first_name \
        FROM customers \
        WHERE first_name LIKE 'J%' ORDER BY first_name")
cur.execute(query)

2

In [4]:
first_name_j = cur.fetchall()

for data in first_name_j:
    print(data)

('Jody',)
('John',)


In [5]:
query2 = ("SELECT * FROM chocolates")
cur.execute(query2)
chocolates_table = cur.fetchall()
for data in chocolates_table:
    print(data)

(1, 'delight_bar', 'milk chocolate waffer nuts', Decimal('3.50'), Decimal('1.12'))
(2, 'protein_choc', 'dark chocolate protein nuts mint', Decimal('5.50'), Decimal('1.90'))
(3, 'white_charm', 'white chocolate nougat waffer', Decimal('4.20'), Decimal('1.35'))
(4, 'caramel_krunch', 'milk chocolate caramel sultanas', Decimal('3.50'), Decimal('1.17'))
(5, 'fam_share_tray', 'assorted milk chocolate tray', Decimal('15.50'), Decimal('6.89'))
(6, 'sr8t_up', 'milk chocolate', Decimal('3.10'), Decimal('0.95'))
(7, 'phils_magnus', 'milk chocolate dark chocolate waffer caramel', Decimal('4.50'), Decimal('1.38'))
(8, 'peppermint_swirl', 'dark chocolate waffer mint', Decimal('3.20'), Decimal('1.15'))
(9, 'choc_attack', 'assorted milk dark white chocolate tray', Decimal('22.75'), Decimal('13.10'))


In [6]:
df = pd.read_sql(query2, db)
df.head()

,product_id,product_name,details,price_aud,cog_aud
0,1,delight_bar,milk chocolate waffer nuts,3.5,1.12
1,2,protein_choc,dark chocolate protein nuts mint,5.5,1.90
2,3,white_charm,white chocolate nougat waffer,4.2,1.35
3,4,caramel_krunch,milk chocolate caramel sultanas,3.5,1.17
4,5,fam_share_tray,assorted milk chocolate tray,15.5,6.89


In [7]:
query3 = ("SELECT DISTINCT \
            order_date, order_id \
        FROM orders \
        WHERE order_date BETWEEN '2023-01-01' AND '2023-01-31'")
df3 = pd.read_sql(query3, db, parse_dates = 'order_date')
df3.head()

,order_date,order_id
0,2023-01-01,OI-01201
1,2023-01-02,OI-01202
2,2023-01-08,OI-01205
3,2023-01-09,OI-01206
4,2023-01-09,OI-01207


In [8]:
# MySQL queries and their conversions to dataframes in Python working correctly.

**STEP 3: Calculate gross profits from MySQL database.**

In [9]:
gpt = ("SELECT \
            o.order_date, \
            o.order_id, \
            o.product_id, \
            ch.price_aud, \
            ch.cog_aud, \
            SUM(ch.price_aud - ch.cog_aud) AS margin, \
            o.quantity, \
            SUM(o.quantity * ch.price_aud) AS total_price, \
            SUM(o.quantity * (ch.price_aud - ch.cog_aud)) AS total_margin \
            FROM orders o \
                JOIN chocolates ch \
                    ON o.product_id = ch.product_id \
                JOIN customers c \
                    ON o.customer_id = c.customer_id \
            GROUP BY 1, 2, 3, 4, 5, 7")
gross_profit = pd.read_sql(gpt, db)
gross_profit.head(5)

,order_date,order_id,product_id,price_aud,cog_aud,margin,quantity,total_price,total_margin
0,2022-01-05,OI-01003,1,3.5,1.12,2.38,3,10.5,7.14
1,2022-01-05,OI-01005,1,3.5,1.12,2.38,1,3.5,2.38
2,2022-01-29,OI-01015,1,3.5,1.12,2.38,1,3.5,2.38
3,2022-02-04,OI-01020,1,3.5,1.12,2.38,2,7.0,4.76
4,2022-02-17,OI-01027,1,3.5,1.12,2.38,3,10.5,7.14


In [10]:
# Inspect data.

gross_profit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_date    1253 non-null   object 
 1   order_id      1253 non-null   object 
 2   product_id    1253 non-null   int64  
 3   price_aud     1253 non-null   float64
 4   cog_aud       1253 non-null   float64
 5   margin        1253 non-null   float64
 6   quantity      1253 non-null   int64  
 7   total_price   1253 non-null   float64
 8   total_margin  1253 non-null   float64
dtypes: float64(5), int64(2), object(2)
memory usage: 88.2+ KB


In [11]:
# Convert order_date field to datetime.

gross_profit = gross_profit.astype({'order_date': 'datetime64'})

In [12]:
gross_profit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_date    1253 non-null   datetime64[ns]
 1   order_id      1253 non-null   object        
 2   product_id    1253 non-null   int64         
 3   price_aud     1253 non-null   float64       
 4   cog_aud       1253 non-null   float64       
 5   margin        1253 non-null   float64       
 6   quantity      1253 non-null   int64         
 7   total_price   1253 non-null   float64       
 8   total_margin  1253 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(2), object(1)
memory usage: 88.2+ KB


In [13]:
print(gross_profit['total_margin'].sum())

19687.059999999998


In [14]:
# Ensure all profit dataframes for dashboard = $19,687.06.
# Dashboard will reflect gross profits as 'Profits' moving forward.

**STEP 4: Create dataframes from MySQL for Python dashboard with checks.**

In [15]:
pbd = ("SELECT \
           o.order_date, \
           SUM(o.quantity * (ch.price_aud - ch.cog_aud)) AS profit \
        FROM orders o \
            JOIN chocolates ch \
                ON o.product_id = ch.product_id \
        GROUP BY 1")
profit_by_date = pd.read_sql(pbd, db)
profit_by_date.sort_values('order_date', inplace = True)
profit_by_date.reset_index(drop = True, inplace = True)
profit_by_date.head()

,order_date,profit
0,2022-01-02,8.55
1,2022-01-05,43.43
2,2022-01-08,3.60
3,2022-01-10,10.80
4,2022-01-13,14.19


In [16]:
profit_by_date = profit_by_date.astype({'order_date': 'datetime64'})

In [17]:
profit_by_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532 entries, 0 to 531
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   order_date  532 non-null    datetime64[ns]
 1   profit      532 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 8.4 KB


In [18]:
# profit_by_date check.

print(profit_by_date['profit'].sum())

19687.059999999998


In [19]:
pbs = ("SELECT \
            c.state, \
            SUM(o.quantity * (ch.price_aud - ch.cog_aud)) AS profit \
        FROM customers c \
            JOIN orders o \
                ON c.customer_id = o.customer_id \
            JOIN chocolates ch \
                ON o.product_id = ch.product_id \
        GROUP BY 1 \
        ORDER BY 2 DESC")
profit_by_state = pd.read_sql(pbs, db)
profit_by_state.head()

,state,profit
0,VIC,10284.67
1,NSW,5143.15
2,QLD,4259.24


In [20]:
# profit_by_state check

print(profit_by_state['profit'].sum())

19687.059999999998


In [21]:
pbg = ("SELECT \
            CASE \
                WHEN gender = 'Male' THEN 'Male' \
                WHEN gender = 'Female' THEN 'Female' \
                ELSE 'Other' \
            END AS gender, \
            SUM(o.quantity * (ch.price_aud - ch.cog_aud)) AS profit \
        FROM customers c \
            JOIN orders o \
                ON c.customer_id = o.customer_id \
            JOIN chocolates ch \
                ON o.product_id = ch.product_id \
        GROUP BY 1 \
        ORDER BY 2 DESC")
profit_by_gender = pd.read_sql(pbg, db)
profit_by_gender.head()

,gender,profit
0,Male,8471.54
1,Female,6697.29
2,Other,4518.23


In [22]:
# profit_by_gender check.

print(profit_by_gender['profit'].sum())

19687.06


In [23]:
pbcity = ("SELECT \
            c.city, \
            SUM(o.quantity * (ch.price_aud - ch.cog_aud)) AS profit \
        FROM customers c \
            JOIN orders o \
                ON c.customer_id = o.customer_id \
            JOIN chocolates ch \
                ON o.product_id = ch.product_id \
            GROUP BY 1 \
            ORDER BY 2")
profit_by_city = pd.read_sql(pbcity, db)
profit_by_city.head()

,city,profit
0,Leongatha,880.06
1,Ipswich,1139.53
2,Tamworth,1606.85
3,Horsham,2609.37
4,Brisbane,3119.71


In [24]:
# profit_by_city check.

print(profit_by_city['profit'].sum())

19687.059999999998


In [25]:
pbp = ("SELECT \
            ch.product_id, \
            ch.product_name, \
            ch.details, \
            SUM(o.quantity * (ch.price_aud - ch.cog_aud)) AS profit \
        FROM orders o \
            JOIN chocolates ch \
                ON o.product_id = ch.product_id \
        GROUP BY 1, 2, 3 \
        ORDER BY 4")
profit_by_product = pd.read_sql(pbp, db)
profit_by_product.head()

,product_id,product_name,details,profit
0,8,peppermint_swirl,dark chocolate waffer mint,928.65
1,6,sr8t_up,milk chocolate,1098.65
2,4,caramel_krunch,milk chocolate caramel sultanas,1453.92
3,1,delight_bar,milk chocolate waffer nuts,1585.08
4,3,white_charm,white chocolate nougat waffer,1633.05


In [26]:
# profit_by_product check.

print(profit_by_product['profit'].sum())

19687.059999999998


In [27]:
# Dataframes ready for analysis.

**STEP 5: Build Dashboard.**

In [28]:
sns.set_style('darkgrid')

fig = plt.figure(figsize = (8, 10), dpi = 1000)
plt.tight_layout()
gs = gridspec.GridSpec(nrows = 32, ncols = 30)
gs.update(hspace = 0.5)

fig.suptitle("PHIL'S CHOCOLATES - PROFIT DASHBOARD",
            fontsize = 18,
            fontweight = 'bold',
            fontfamily = 'calibri',
            color = '#5E2612')

ax1 = fig.add_subplot(gs[0:5, 0:])
ax2 = fig.add_subplot(gs[8:14, 0:14])
ax3 = plt.subplot(gs[8:14, 16:])
ax4 = plt.subplot(gs[16:32, 0:13])
ax5 = plt.subplot(gs[16:32, 17:])

ax1.scatter(x = 'order_date',
            y = 'profit',
            color = 'limegreen',
            marker = '.',
            s = 14,
            data  = profit_by_date
           )

ax1.set_title('Profit by Date',
              fontsize = 12,
              fontweight = 'heavy',
              color = 'indigo'
             )

ax1.set_xlabel('Order Date',fontsize = 8, color = 'darkred')
ax1.set_ylabel('Profit Total $', fontsize = 8, color = 'darkred')
ax1.set_xticklabels(['Jan-22', 'May-22', 'Aug-22', 'Jan-23', 'May-23', 'Aug-23', 'Jan-24', 'May-24', 'Aug-24', 'Jan-25'],
                    fontsize = 8)
ax1.set_yticks(ticks = [0, 100, 200], labels = [0, 100, 200], fontsize = 8)

ax2.pie(x = profit_by_state['profit'],
        labels = profit_by_state['state'],
        colors = ['dodgerblue', 'limegreen', 'gold'],
        wedgeprops = {'edgecolor': 'black'},
        explode = [0.09, 0, 0],
        autopct = '%1.1f%%',
        startangle = 60,
        labeldistance = 1.20,
        textprops = {'size': 'small'},
        radius = 1.1
        )

ax2.set_title('Profit By State',
              fontsize = 12,
              fontweight = 'heavy',
              color = 'indigo')

ax3.pie(x = profit_by_gender['profit'],
        labels = profit_by_gender['gender'],
        colors = ['dodgerblue', 'limegreen', 'gold'],
        wedgeprops = {'edgecolor': 'black'},
        explode = [0.09, 0, 0],
        autopct = '%1.1f%%',
        startangle = 70,
        labeldistance = 1.20,
        textprops = {'size': 'small'},
        radius = 1.1
        )

ax3.set_title('Profit By Gender',
              fontsize = 12,
              fontweight = 'heavy',
              color = 'indigo'
             )

ax4.bar(x = 'city',
        bottom = 800,
        height = 'profit',
        width = 0.5,
        data = profit_by_city,
        edgecolor = 'black',
        color = ['dodgerblue', 'limegreen', 'gold', 'dodgerblue', 'limegreen', 'gold','dodgerblue']
        )

ax4.set_title('Profit by City',
              fontsize = 12,
              fontweight = 'heavy',
              color = 'indigo'
             )

ax4.set_xlabel('City', fontsize = 8, color = 'darkred')
ax4.set_ylabel('Profit Total $', fontsize = 8, color = 'darkred')
ax4.set_xticklabels(['Leo', 'Ips', 'Tam', 'Hor', 'Bri', 'Syd', 'Mel'], fontsize = 8)
ax4.set_yticklabels(['1,000', '2,000', '3,000', '4,000', '5,000', '6,000', '7,000', '8,000'], fontsize = 8)

ax5.barh(y = 'product_name',
         width = 'profit',
         height = 0.5,
         data = profit_by_product,
         edgecolor = 'black',
         color = ['dodgerblue', 'limegreen', 'gold', 'dodgerblue', 'limegreen', 'gold','dodgerblue', 'limegreen', 'gold']
        )

ax5.set_title('Profit by Product',
              fontsize = 12,
              fontweight = 'heavy',
              color = 'indigo'
             )

ax5.set_xlabel('Profit Total $',fontsize = 8, color = 'darkred')
ax5.set_ylabel('Product', fontsize = 8, color = 'darkred')

ax5.set_yticklabels(['SWL', 'S8U', 'CKR', 'DEB', 'WCH', 'PHM', 'PCH', 'CHA', 'FMT', ''], fontsize = 8)
ax5.set_xticklabels(['0', '1,000', '2,000', '3,000', '4,000', '5,000'], fontsize = 8)

plt.show();


fig.savefig("C:\\Users\\Phil\\Documents\\PHELUCIAM\\GENERAL\\JOB\\CAREER DEVELOPMENT\\DATA SETS\\PHILS CHOCOLATES\\python_mySQL_dashboard.jpeg",
            dpi = 1000,
            bbox_inches = 'tight',
            facecolor = 'white')

**STEP 6: Conculsions.**

*(a) Observations.*

In [29]:
# Phil's Chocolates have struggled with profits in years 2022 & 2023.
# Year 2024 has shown significant improvement, however the business still struggles to grow sustainably.
# The vast majority of profits are driven within Victoria, followed by NSW with QLD close behind.
# Males account for the vast majority of profits, followed by females.
# Melbourne is far and away the frontrunner in regards to generating profit for the company.
# The Family Share Tray and Choc Attack Tray drive the most profits.
# The Peppermint Swirl and Sr8t Up bars generate the lowest profits for the business.

*(b) Recommendations.*

In [30]:
# A sales team in NSW and QLD to generate more business could be considered.
# Promotions such as minimum buy discounts to existing female customers maybe beneficial.
# Social Media campaigns promoting the Peppermint Swirl & Str8t Up bars may generate more sales and thus profits.
# Discussions with suppliers regarding cost of goods (cogs) could prove adventageous.

**Appendicies.**

In [31]:
# This project is designed to demonstrate how to write MySQL queries in Python to create a Python Dashboard.
# However, please note dataframes can be created in Python from original MySQL dataframe gross_profit.

In [32]:
gross_profit.head()

,order_date,order_id,product_id,price_aud,cog_aud,margin,quantity,total_price,total_margin
0,2022-01-05,OI-01003,1,3.5,1.12,2.38,3,10.5,7.14
1,2022-01-05,OI-01005,1,3.5,1.12,2.38,1,3.5,2.38
2,2022-01-29,OI-01015,1,3.5,1.12,2.38,1,3.5,2.38
3,2022-02-04,OI-01020,1,3.5,1.12,2.38,2,7.0,4.76
4,2022-02-17,OI-01027,1,3.5,1.12,2.38,3,10.5,7.14


*(i) Profit by order_date using the groupby method from the Pandas Library as an example.*

In [33]:
pandas_date_profit_groupby = pd.DataFrame(gross_profit.groupby(['order_date'])['total_margin'].sum())
pandas_date_profit_groupby.head()

,total_margin
order_date,
2022-01-02,8.55
2022-01-05,43.43
2022-01-08,3.60
2022-01-10,10.80
2022-01-13,14.19


In [34]:
# pandas_date_profit_groupby check.

print(pandas_date_profit_groupby['total_margin'].sum())

19687.059999999998


*(ii) Profit by order_date & order_id using the pivot method from the Pandas Library as an example.*

In [35]:
pandas_date_profit_pivot = pd.pivot_table(gross_profit, index = ['order_date', 'order_id'],
                                          values = ['total_margin'], aggfunc = 'sum')
pandas_date_profit_pivot.head()

total_margin
order_date order_id              
2022-01-02 OI-01001          8.55
2022-01-05 OI-01002          8.61
           OI-01003         14.13
           OI-01004          8.26
           OI-01005          5.23

In [36]:
pandas_date_profit_pivot = pandas_date_profit_pivot.rename(columns = {'total_margin': 'profit'})
pandas_date_profit_pivot.head()

profit
order_date order_id        
2022-01-02 OI-01001    8.55
2022-01-05 OI-01002    8.61
           OI-01003   14.13
           OI-01004    8.26
           OI-01005    5.23

In [37]:
# pandas_date_profit_pivot check.

print(pandas_date_profit_pivot['profit'].sum())

19687.059999999998
